# Desafío Alkemy

## Challenge Data Analytics -Python

### Objetivo
Crear un proyecto que consuma datos desde 3 fuentes distintas para popular una base de datos SQL con información cultural sobre bibliotecas, museos y salas de cines argentinos.

#### 0. Importar librerías

In [1]:
from bs4 import BeautifulSoup
import requests
import os
from datetime import date
import pandas as pd

#### 1.a Conseguir URLs de los csv
Obtener los 3 archivos de fuente utilizando la librería requests y almacenarse en forma local<br> 
(Ten en cuenta que las urls pueden cambiar en un futuro):
- Datos Argentina - Museos
- Datos Argentina - Salas de Cine
- Datos Argentina - Bibliotecas Populares

In [2]:
url = 'https://datos.cultura.gob.ar/dataset/espacios-culturales-argentina-sinca'


In [4]:
r = requests.get(url)
contenido = r.text
soup = BeautifulSoup(r.text, 'lxml')

In [9]:
def get_urls(url):
    '''
    Devuelve un diccionario con las urls de los recursos del dataset
    Pasar como argumento la url del dataset
    '''
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'lxml')
    diccionario = {}
    for i in soup.find_all('h3'):
        diccionario[i.text.strip()] = soup.find('h3', string=i.text)\
                             .find_previous('button', string='DESCARGAR')\
                             .find_previous('a')['href']
    return diccionario
dataset = get_urls(url)

In [11]:
dataset

{'Bibliotecas Especializadas': 'https://datos.cultura.gob.ar/dataset/37305de4-3cce-4d4b-9d9a-fec3ca61d09f/resource/456d1087-87f9-4e27-9c9c-1d9734c7e51d/download/08_biblio_espec.xlsx-sheet2.csv',
 'Bibliotecas Populares': 'https://datos.cultura.gob.ar/dataset/37305de4-3cce-4d4b-9d9a-fec3ca61d09f/resource/01c6c048-dbeb-44e0-8efa-6944f73715d7/download/biblioteca_popular.csv',
 'Casas del Bicentenario': 'https://datos.cultura.gob.ar/dataset/37305de4-3cce-4d4b-9d9a-fec3ca61d09f/resource/8d0b7f33-d570-4189-9961-9e907193aebc/download/casas-del-bicentenario.csv',
 'Museo': 'https://datos.cultura.gob.ar/dataset/37305de4-3cce-4d4b-9d9a-fec3ca61d09f/resource/4207def0-2ff7-41d5-9095-d42ae8207a5d/download/museos_datosabiertos.csv',
 'Sala de Teatro': 'https://datos.cultura.gob.ar/dataset/37305de4-3cce-4d4b-9d9a-fec3ca61d09f/resource/87ebac9c-774c-4ef2-afa7-044c41ee4190/download/17_teatro.xlsx-datos-abiertos.csv',
 'Monumentos y lugares históricos': 'https://datos.cultura.gob.ar/dataset/37305de4-3cc

#### 1.b Guardar archivos
Organizar los archivos en rutas siguiendo la siguiente estructura: <br> **“categoría\año-mes\categoria-dia-mes-año.csv”**
- Por ejemplo: “museos\2021-noviembre\museos-03-11-2021”
- Si el archivo existe debe reemplazarse. La fecha de la nomenclatura es la fecha de descarga.

In [12]:
# Definir strings de fechas
anio_mes = date.today().strftime('%Y-%B').lower()
dia_mes_anio = date.today().strftime('%d-%m-%Y')

In [13]:
anio_mes

'2023-may'

In [6]:
# Definir directorios
dir_actual = os.getcwd()
dir_bibliotecas = os.path.join(dir_actual+'/bibliotecas/'+anio_mes)
dir_museos = os.path.join(dir_actual+'/museos/'+anio_mes)
dir_cines = os.path.join(dir_actual+'/cines/'+anio_mes)

In [7]:
# Creando directorio
def crear_dir(dir_categoria):
    if not os.path.exists(dir_categoria):
            os.makedirs(dir_categoria)

In [8]:
crear_dir(dir_bibliotecas)
crear_dir(dir_cines)
crear_dir(dir_museos)

In [9]:
# Definir urls para cada base de datos
url_bibliotecas = dataset.get('Bibliotecas Populares')
url_museos = dataset.get('Museo')
url_cines = dataset.get('Cine')


In [10]:
# Definir path de archivos descargados
csv_bibliotecas = os.path.join(dir_bibliotecas+'/bibliotecas-'+dia_mes_anio+'.csv')
csv_museos = os.path.join(dir_museos+'/museos-'+dia_mes_anio+'.csv')
csv_cines = os.path.join(dir_cines+'/cines-'+dia_mes_anio+'.csv')

In [11]:
# Descargando urls
def descargar_csv(url_csv, dir_salida):
    response = requests.get(url_csv)
    with open(dir_salida, 'wb') as csv:
        csv.write(response.content)

In [12]:
descargar_csv(url_bibliotecas, csv_bibliotecas)
descargar_csv(url_museos, csv_museos)
descargar_csv(url_cines, csv_cines)